# Connecting to free language learing model APIs

## 1. Co:here

In [1]:
# import libraries
import pandas as pd
import cohere
from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', None)

In [11]:
# get the data
df = pd.read_excel("../data/Example_data.xlsx")

In [13]:
# look at data
df.head(1)

Domain  \
0  rassegnastampa.news   

                                                                                                                       Title  \
0  Boris Johnson using a taxpayer-funded jet for an election campaign fits a long history of taking things he didn't pay for   

                                                                                                                                                                                                                                                   Description  \
0  …often trigger a protest vote that can upset…that ministers who breach the Ministerial Code…plane up the M1 in order to…serious questions about abuse of taxpayers' money which is a clear breach of the code.…should not permit abuse of taxpayers' money…   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [14]:
df.shape

(10, 9)

In [17]:
# df.index()

In [20]:
# create a sample dataset
num_examples = 5
df_sample = df.sample(num_examples)

In [22]:
# split into training and testing sets
# we set the 'Body' column as the prompt and the 'Analyst_Average_Score' col as the target variable
sentences_train, sentences_test, labels_train, labels_test = train_test_split(list(df_sample['Body']), 
                                                                list(df_sample['Analyst_Average_Score']), 
                                                                test_size=0.25, 
                                                                random_state=0)

### Getting the embeddings of the item content

In [48]:
# connecting to the api
api_key = "MI6N12pjjldS65peoYcrjTUBegNDGLWPz2t5OIU9" # generated a new api key while i figure out how to use secrets in the yml file

# creating and retrieving a cohere API key from os.cohere.ai
co = cohere.Client(api_key)

In [49]:
# embed the training set
embeddings_train = co.embed(texts=sentences_train,
                             model="large",
                             truncate="LEFT").embeddings
# embed the testing set
embeddings_test = co.embed(texts=sentences_test,
                             model="large",
                             truncate="LEFT").embeddings

We have not created 2 sets of embeddings:
<li> embeddings_train - contains embeddings of the training sentences
<li> embeddings_test - contains embeddings of the testing sentences

In [50]:
# to see what an embedding looks like
print(f"Review text: {sentences_train[0]}")
print(f"Embedding vector: {embeddings_train[0][:10]}")

Review text: Boris Johnson using a taxpayer-funded jet for an election campaign fits a long history of taking things he didn't pay for A biographer of Johnson and a former colleague said that Johnson's flight to Hartlepool in an election campaign is totally in character. - Boris Johnson flew in a tax-funded private jet to campaign in the Hartlepool by-election. - The trip may have breached ministerial rules, though officials are reluctant to investigate. - Skirting rules and taking freebies is typical of Johnson, as those close to him have noted. - [See more stories on Insider's business page] . Evacuating yourself into Afghanistan is an unusual move, but one that Boris Johnson made anyway. In June 2018, on the day of a crucial vote on the expansion of London's Heathrow airport, the then-foreign secretary discovered he had urgent business in Kabul, placing him more than 3,500 miles from parliament. Had Johnson been in Westminster, he would have faced a choice: follow Prime Minister The

### Training a classifier using the training set

In [51]:
# # import SVM classifier code
# from sklearn.svm import SVC
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import StandardScaler


# # Initialize a support vector machine, with class_weight='balanced' because 
# # our training set has roughly an equal amount of positive and negative 
# # sentiment sentences
# svm_classifier = make_pipeline(StandardScaler(), SVC(class_weight='balanced')) 

# # fit the support vector machine
# svm_classifier.fit(embeddings_train, labels_train)

### Evaluate the performance of the classifier on the testing set

In [52]:
# get the score from the test set, and print it out to screen!
# score = svm_classifier.score(embeddings_test, labels_test)
# print(f"Validation accuracy on Large is {100*score}%!")

## 2. DeepAI
The API we will try to use will be the Text to Image API. <br>
This API creates an image from scratch from a text description

In [30]:
# importing libraries

# Example posting a text URL:

import requests
r = requests.post(
    "https://api.deepai.org/api/text2img",
    data={
        'text': 'Example: posting a text URL',
    },
    headers={'api-key': '7f2f6ce1-98c4-404d-9e6f-ce0a8e65ea72'}
)
print(r.json())


# Example posting a local text file:

# import requests
# r = requests.post(
#     "https://api.deepai.org/api/text2img",
#     files={
#         'text': open('/path/to/your/file.txt', 'rb'),
#     },
#     headers={'api-key': '7f2f6ce1-98c4-404d-9e6f-ce0a8e65ea72'}
# )
# print(r.json())


# Example directly sending a text string:

import requests
r = requests.post(
    "https://api.deepai.org/api/text2img",
    data={
        'text': 'Example: directly sending a text string',
    },
    headers={'api-key': '7f2f6ce1-98c4-404d-9e6f-ce0a8e65ea72'}
)
print(r.json())

{'status': 'Out of free credits - please enter payment info in your dashboard: https://deepai.org/dashboard'}
{'status': 'Out of free credits - please enter payment info in your dashboard: https://deepai.org/dashboard'}


Requires payment information for the api to work. Therefore we will skip this API for now.

## 3. Muse by LightOn

Requires payment info to be able to use the api as well. We'll skip this one as well.

## 4. BLOOM
Using the huggingface space. 